### Imports and Installations Required for Generating Music

In [26]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import random
from glob import glob
from collections import defaultdict
from numpy.random import choice
from symusic import Score
from miditok import REMI, TokenizerConfig
from midiutil import MIDIFile
import mido

### Extract (pitch, duration) tuples from a MIDI file.

In [27]:
def midi_to_note_duration_sequence(filename):
    mid = mido.MidiFile(filename)
    notes = []
    abs_time = 0
    note_on_times = {}
    for msg in mid:
        abs_time += msg.time
        if msg.type == 'note_on' and msg.velocity > 0:
            note_on_times[msg.note] = abs_time
        elif (msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0)) and msg.note in note_on_times:
            duration = abs_time - note_on_times[msg.note]
            notes.append((msg.note, int(duration * 480)))  # scale duration for MIDI ticks
            del note_on_times[msg.note]
    return notes

### Create a MIDI file from a sequence of notes, assigning a fixed duration (note_length) to all notes. 

In [28]:
def note_sequence_to_midi(note_sequence, filename, velocity=64, tempo=500000, note_length=120):
    mid = mido.MidiFile()
    track = mido.MidiTrack()
    mid.tracks.append(track)
    track.append(mido.MetaMessage('set_tempo', tempo=tempo))
    for note in note_sequence:
        track.append(mido.Message('note_on', note=note, velocity=velocity, time=0))
        track.append(mido.Message('note_off', note=note, velocity=velocity, time=note_length))
    mid.save(filename)

### Generate a MIDI file with custom durations for each note, allowing variable-length notes.

In [29]:
def note_duration_sequence_to_midi(note_duration_sequence, filename, velocity=64, tempo=500000):
    mid = mido.MidiFile()
    track = mido.MidiTrack()
    mid.tracks.append(track)
    track.append(mido.MetaMessage('set_tempo', tempo=tempo))
    for note, duration in note_duration_sequence:
        track.append(mido.Message('note_on', note=note, velocity=velocity, time=0))
        track.append(mido.Message('note_off', note=note, velocity=velocity, time=duration))
    mid.save(filename)

Random seed can be changed to get different results, default its 42, from CSE_153R Homework 3(Spring 2025)

In [30]:
random.seed(42)

### Loading the music data

In [31]:
#first is popular pop songs I enjoy
midi_files = glob('data/*.mid')
#same midi file data We have trained on for Assigment 1 Task 1
#midi_files = glob('Assignment1(Task1_midis)/*.midi')
len(midi_files)

8